In [1]:
from methods.llm import get_async_vllm_endpoint
import os 

# Unlimited LLM endpoints
endpoint_id = "vllm-8sz1f7zg7oy0ui"
api_key = "rpa_EPOJED42G59S80Y6SKMCOI330EQU4JPPMKV2UD2W7j0uku"
get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key)

Could not load vllm class, check CUDA support and GPU RAM size


#### Temasek Foundation Dataset

In [2]:
# Node dataset should ideally be concise and pure application info included ...
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.load_data import prep_tf_node 

tf_meta_prompt, test_cases, tf_metric_map = prep_tf_node(prompt_mode=True)

# Specific Metric required for TF dataset (Worth refactor the code a bit)
from methods.llm import get_groq_response
from methods.evolnode import EvolNode 
node = EvolNode(tf_meta_prompt, None, None, get_response=get_endpoint_response, test_cases=test_cases[:90],
                custom_metric_map=tf_metric_map) # setting manual test cases


In [3]:
node.evolve("i1", batch_size=5)

     :: Query time: 23.15s


Processing LLM queries: 100%|██████████| 5/5 [00:13<00:00,  2.71s/it]


 :: Total time elapsed: 13.54s, 0 errors
     :: Evolution time: 13.59s


Processing LLM queries:   0%|          | 1/1350 [00:10<3:58:12, 10.59s/it]

Error in completion: 'NoneType' object has no attribute 'choices'
Error in completion: 'NoneType' object has no attribute 'choices'
Error in completion: 'NoneType' object has no attribute 'choices'
Error in completion: 'NoneType' object has no attribute 'choices'
Error in completion: 'NoneType' object has no attribute 'choices'
Error in completion: 'NoneType' object has no attribute 'choices'
Error in completion: 'NoneType' object has no attribute 'choices'
Error in completion: 'NoneType' object has no attribute 'choices'
Error in completion: 'NoneType' object has no attribute 'choices'


...

Processing LLM queries: 100%|██████████| 1350/1350 [02:42<00:00,  8.29it/s]


 :: Total time elapsed: 162.80s, 27 errors

Code 0 outputs:
Test 0: {'label': 'No', 'comment': "The grant application lacks a clear project plan, scope of work, and objectives, making it difficult to evaluate the project's feasibility and potential impact."}
Test 2: {'label': 'No', 'comment': "The grant application lacks essential information and does not provide a clear project scope, objectives, and proof of concept. Although the project idea is innovative and addresses a significant issue, the application does not demonstrate a thorough understanding of the project's feasibility, budget, and evaluation plan."}
Errors: ['Failed to parse LLM response -- No JSON structure found in the provided text.']
Test 3: {'label': 'Maybe', 'comment': 'The grant application is incomplete, and the proposed solution lacks specific objectives, scope of work, and proof of concept. However, the idea of utilizing waste cooking oil and shrimp shells to create a sustainable and biodegradable lubricant show

Processing LLM queries: 100%|██████████| 724/724 [01:04<00:00, 11.23it/s] 


 :: Total time elapsed: 64.48s, 0 errors


ZeroDivisionError: division by zero

In [11]:
input_dicts = test_cases[:90]



from tqdm import tqdm 
import types 
from methods.meta_execute import Timeout_
from collections import defaultdict

code = 'def generate_prompt(project_description):\n    decision = \'Maybe\'\n    comment = \'No reasonable decision can be made without further information.\'\n    prompt = (\n        f"Given the project description {{project_description= \'{project_description}\' }}, do evaluate the grant application, make a decision (Yes, No, Maybe) and a brief comment explaining why the project is likely to be accepted or rejected. Make sure the output is in JSON format in markdown like this {{\'label\': \'str()\', \'comment\': \'str()\'}} near the top of your response."\n        )\n    return prompt'
codes = [code]
max_tries = 3


prompts = []
input_indices = []
errors_per_code_per_test = defaultdict(lambda: defaultdict(list))

total_iterations = len(input_dicts) * len(codes)
with tqdm(total=total_iterations, desc="Executing Node to generate prompts ...") as pbar:
    for (input_index, input_dict) in enumerate(input_dicts):
        for (code_index, code) in enumerate(codes):
            mod = types.ModuleType('dynamic_module')
            try:
                with Timeout_(3):  # 3-second timeout
                    exec(code, mod.__dict__)
                    func_name = "generate_prompt"
                    assert func_name in mod.__dict__, f"Function {func_name} not found in code #{code_index}"
                    prompt_func = mod.__dict__[func_name]
                    prompt = prompt_func(**input_dict)
                    
                    for _ in range(max_tries): # number of trials
                        prompts.append(prompt)
                        input_indices.append((input_index, code_index))
                        
                errors_per_code_per_test[code_index][input_index] = [] # empty error when one trial works 
                        
            except Exception as e:
                errors_per_code_per_test[code_index][input_index].append(str(e))
            
            pbar.update(1)

Executing Node to generate prompts ...: 100%|██████████| 90/90 [00:00<00:00, 28752.18it/s]
